In [11]:
import asyncio
from playwright.async_api import async_playwright
import aiohttp
import os

async def download_image(session, url, filename):
    async with session.get(url) as resp:
        if resp.status == 200:
            with open(filename, 'wb') as f:
                f.write(await resp.read())
                
async def crawl_tours():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # True để khỏi bật browser
        page = await browser.new_page()

        await page.goto("https://hanoitourist.vn/tour-trong-nuoc")
        await page.wait_for_timeout(8000)  # Chờ các lazyload tải xong

        # Lấy từng khối tour
        tour_cards = await page.locator("div.tour-item").all()
        async with aiohttp.ClientSession() as session:
            for i, card in enumerate(tour_cards):
                title = await card.locator("h3 a").inner_text()
                price = await card.locator("div.detail-gia").inner_text()
                location = await card.locator("div.detail-item-value >> nth=1").inner_text()
                dates = await card.locator("div.detail-item-value >> nth=0").inner_text()
                image_url = await card.locator("div.tour-img img").get_attribute("src")
                filename = f"tour_image_{i+1}.jpg"

            # Bỏ qua nếu không có ảnh
                if image_url:
                    await download_image(session, image_url, filename)
                print(f"🎯 Tour {i+1}: {title}")
                print(f"📍 {location} | 📅 {dates} | 💰 {price}")
                print(image_url)
                print("────────────")
                if i == 0:
                    break
            await browser.close()

# Nếu chạy trong notebook thì dùng nest_asyncio
import nest_asyncio
nest_asyncio.apply()
await crawl_tours()




🎯 Tour 1: THÁC DATANLA – VINPEARL NHA TRANG – VỊNH SAN HÔ – LÀNG CHÀI – BÃI TRANH
📍 Ngày đi: 17/04, 08/05, 15/05/2025... (4 ngày 3 đêm) | 📅 Hành trình: Đà Lạt - Nha Trang - 4 ngày 3 đêm | 💰 7,250,000đ
https://hanoitourist.vn/sites/default/files/2024/02/1_2.png
────────────
